# TOMI INDEX

Para realizar a interpolação do TOMI Index, insira abaixo o caminho do arquivo xlsx, xls, ou csv.

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import openpyxl
import scienceplots

plt.style.use(["science", "bmh", "notebook", "grid"])

file_name = "./uploads/@CN_OGX-49.xlsx"

columns = input("Digite no format A:Z de qual coluna até qual coluna você quer extrair o dataframe: ")
pagina = input("Digite o nome da página da planilha em que estão os dados")

dataframe = pd.read_excel(
    file_name, skiprows=0, usecols=columns, sheet_name=pagina
)

dataframe

,Profundidade,Remessa,TN%,d15N,TOC%,d13C,TOC/TN
0,1665,PlacaSolosB1,0.023717,NaN,0.674199,-27.204858,28.426833
1,1674,NaN,0.049761,-1.999375,1.138534,-26.903999,22.880187
2,1677,NaN,0.041436,NaN,1.014081,-27.134068,24.473313
3,1680,NaN,0.056889,0.514699,0.612316,-25.406586,10.763327
4,1683,NaN,0.022852,NaN,0.459156,-26.144969,20.092525
5,1686,NaN,0.044987,-0.778253,0.770819,-26.520551,17.134109
6,1689,NaN,0.042003,NaN,0.576998,-25.773319,13.736999
7,1692,NaN,0.070471,-0.656849,1.032965,-26.360290,14.657972
8,1695,NaN,0.080638,1.091368,0.990373,-25.904085,12.281701
9,1698,NaN,0.096140,1.539552,1.388665,-26.490072,14.444255


In [17]:
import numpy as np
from pykrige.ok import OrdinaryKriging
import matplotlib.pyplot as plt

# Dados dos pontos-chave de probabilidade
art_toc_tn = np.array(
    [4, 4, 4, 4, 10, 10, 10, 10, 100, 100, 100, 100]
)  # Lista/array dos valores de TOC:TN dos pontos-chave
art_c13corg = np.array(
    [-10, -22, -25, -34, -10, -22, -25, -34, -10, -22, -25, -34]
)  # Lista/array dos valores de δ13Corg dos pontos-chave
probabilidade = np.array(
    [0, 10, 20, 30, 20, 30, 40, 50, 90, 95, 98, 100]
)  # Lista/array dos valores de probabilidade dos pontos-chave

# Dados das suas amostras
amostra_art_toc_tn = TOC_TN  # Lista/array dos valores de TOC:TN das suas amostras
amostra_art_c13corg = d13C  # Lista/array dos valores de δ13Corg das suas amostras

# Grade de valores para interpolação
art_toc_tn_grid = np.linspace(0, 100, 1001)  # Valores de TOC:TN para a grade
art_c13corg_grid = np.linspace(-34, -10, 241)  # Valores de δ13Corg para a grade

# Kriging
OK = OrdinaryKriging(
    art_toc_tn,
    art_c13corg,
    probabilidade,
    variogram_model="gaussian",
)
z, ss = OK.execute("grid", art_toc_tn_grid, art_c13corg_grid)

# Aplicação dos dados das suas amostras na grade interpolada
amostra_probabilidade, ss = OK.execute(
    "points", amostra_art_toc_tn, amostra_art_c13corg
)

print(np.shape(z))
print(np.shape(amostra_probabilidade))

# scatter = plt.scatter(amostra_art_toc_tn, amostra_art_c13corg, c=amostra_probabilidade, cmap='viridis', s=30, label='Artigo')
# plt.xlabel('TOC:TN')
# plt.ylabel('delta13C')
# # plt.xlim(0,1)
# plt.ylim(-34,-20)
# cbar = plt.colorbar(scatter)
# cbar.set_label('tomi index')
# plt.show()

(241, 1001)
(32,)


In [ ]:
plt.figure(figsize=(4, 8))

plt.plot(amostra_probabilidade, profundidade, "s-", label="TOMI")
plt.gca().invert_yaxis()
plt.xlim(0, 100)
plt.title("TOMI Index")
plt.xlabel("Index (%)")

In [22]:
import pandas as pd

fmane2 = "./uploads/@CN_OGX-49.xlsx"

longa = pd.read_excel(fmane2, skiprows=0, sheet_name="TUDO")

longa.iloc[:, 0]

0     1665
1     1674
2     1677
3     1680
4     1683
5     1686
6     1689
7     1692
8     1695
9     1698
10    1701
11    1704
12    1707
13    1710
14    1713
15    1716
16    1719
17    1722
18    1725
19    1728
20    1731
21    1734
22    1737
23    1740
24    1743
25    1746
26    1749
27    1752
28    1755
29    1758
30    1761
31    1764
Name: Profundidade, dtype: int64